<a href="https://colab.research.google.com/github/sarab421/Sales-Person-Chatbot/blob/main/chatWithYourData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
!pip install langchain_groq
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os
import asyncio  # Import asyncio for running the event loop
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = getpass.getpass()

model = ChatGroq(model="llama3-8b-8192")

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a best assistant. Answer all questions to the best of your ability. If you don't know the answer simply say i don't know otherwise answer it",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Async function for node:
async def call_model(state: MessagesState):
    chain = prompt | model
    response = await chain.ainvoke(state["messages"])
    return {"messages": response}

# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# This needs to be inside an async function

query = "Hi! My name is Sarab?."
config = {"configurable": {"thread_id": "abc123"}}
input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()



··········
================================== Ai Message ==================================

Hello Sarab! It's nice to meet you!


In [ ]:
query = """Can you summarize: In lands where deserts kissed the sky, /n
A humble heart was born to rise,
Not for himself, nor crown or gold,
But for the tale of justice told.

With wisdom deep and courage bright,
He rode the winds, embraced the fight.
In ancient streets, where echoes rang,
A hero's name in honor sang.

Jerusalem, the sacred prize,
Shone in the tear of faithful eyes,
Yet Salahuddin, with mercy’s hand,
Chose peace to heal a broken land.

No tyrant’s greed, no warrior's pride,
Could ever sway him from the side
Of truth, of grace, of noble aim—
His was a light, a guiding flame.

A knight of faith, with banner high,
Who knew when swords should cease to fly,
And in his name, the world would see,
A legacy of unity.

Though time may fade the stones and sand,
Salahuddin’s mark will stand,
In every heart where honor thrives,
His story lives, his spirit strives."""

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

This is a poem about the life and legacy of Salahuddin (also known as Saladin), a 12th-century Muslim leader who played a key role in the recapture of Jerusalem from the Crusaders.

The poem describes how Salahuddin was born with a humble heart, driven by a desire for justice and a commitment to telling the story of his people. It highlights his bravery, wisdom, and compassion, as well as his refusal to be swayed by personal gain or pride.

The poem also mentions Salahuddin's role in choosing peace over conflict, and his efforts to heal a broken land. It describes him as a knight of faith who knew when to lay down his sword, and whose legacy is one of unity and honor.

Finally, the poem notes that while the physical stones and sand may fade with time, Salahuddin's mark will remain, inspiring future generations to strive for honor and unity.


In [ ]:

query = """In desert sands, where banners flew,
A warrior bold, a heart so true.
With courage vast and eyes afire,
Khalid rode, a force entire.

Once his sword was drawn in might,
He knew no fear, he feared no fight.
The winds of war sang out his name,
In victory’s glow, he earned his fame.

He stood for truth with firm resolve,
In battles fierce, his strength evolved.
From Uhud's field to Yarmouk's plain,
He led with honor, conquered pain.

The "Sword of Allah," a title grand,
Bestowed on him by divine command.
For every foe that faced his blade,
Would see their will to fight soon fade.

But not just steel, his wisdom bright,
Shaped the course of day and night.
A leader strong, a guide so wise,
Whose valor reached the boundless skies.

Though battles waged, his heart was still,
A servant true to Allah’s will.
And in the pages of time he stays,
A legend told through endless days.

So sing of Khalid, fierce and free,
A symbol of bravery’s legacy.
His name endures, his deeds remain,
A warrior blessed in heaven's domain."""

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()




================================== Ai Message ==================================

This is a poem about the life and legacy of Khalid ibn al-Walid, a companion of the Prophet Muhammad and a renowned warrior in the early Islamic conquests.

The poem describes Khalid as a bold and true warrior, with a heart full of courage and a will to fight for what is right. It highlights his bravery in battle, his unwavering commitment to truth, and his leadership skills, which earned him the title "Sword of Allah".

The poem also mentions Khalid's many victories, including the battles of Uhud and Yarmouk, and his ability to inspire his troops and lead them to victory. It notes that Khalid's wisdom and guidance went beyond mere military prowess, and that he was a wise and just leader who served as a role model for his followers.

The poem concludes by praising Khalid as a symbol of bravery and legacy, and notes that his name and deeds will endure for all eternity. It also mentions that Khalid's legacy

## OK now we will tbe using our own Data

For that we will first load our Data

In [ ]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "/content/sample_data/sample.csv"
)

loader = CSVLoader(file_path=file_path)
data = loader.load()

for record in data[:2]:
    print(record)

page_content='title: Audi A4
price: 10000000
year: 2016
mileage: 92000
fuel_type: Petrol
horsepower: 2000
transmission: Automatic
location: Lahore
registration: Islamabad
color: Black
dealer: Amir
listing_date: Oct 15, 2024
features: ABS,  AM/FM Radio,  Air Bags,  Air Conditioning,  Alloy Rims,  CD Player,  Cruise Control,  DVD Player,  Immobilizer Key,  Keyless Entry,  Navigation System,  Power Locks,  Power Mirrors,  Power Steering,  Power Windows,  Sun Roof
link: https://www.pakwheels.com/used-cars/audi-a4-2016-for-sale-in-lahore-9085266
images: ['670e55873c76c40f756a89b8']
page: page 1
_id: 670e55873c76c40f756a89c1
description: Audi A4, Petrol, Automatic, Lahore, Islamabad, Black, ABS,  AM/FM Radio,  Air Bags,  Air Conditioning,  Alloy Rims,  CD Player,  Cruise Control,  DVD Player,  Immobilizer Key,  Keyless Entry,  Navigation System,  Power Locks,  Power Mirrors,  Power Steering,  Power Windows,  Sun Roof' metadata={'source': '/content/sample_data/sample.csv', 'row': 0}
page_cont

For Vector Store we need to create embeddings of our data for that:

In [ ]:
pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings= embeddings_model.embed_documents([doc.page_content for doc in data])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings

[[-0.027836959809064865,
  0.042225901037454605,
  -0.0214228555560112,
  0.07270313054323196,
  0.009720227681100368,
  -0.0065128374844789505,
  0.01872619427740574,
  0.063124880194664,
  -0.03866078704595566,
  0.014429599046707153,
  -0.046090174466371536,
  0.014853854663670063,
  0.05082051455974579,
  0.042681124061346054,
  -0.020917220041155815,
  0.018892966210842133,
  -0.0017249644733965397,
  0.01472581084817648,
  0.025435641407966614,
  -0.06675167381763458,
  -0.03309721499681473,
  -0.013304665684700012,
  -0.003846699371933937,
  -0.00979117676615715,
  -0.03502589464187622,
  0.03206942602992058,
  0.032534729689359665,
  -0.08154015988111496,
  -0.011077707633376122,
  -0.03303813189268112,
  -0.011800936423242092,
  0.023097822442650795,
  0.03956654667854309,
  0.03252766281366348,
  1.9550113847799366e-06,
  -0.037013061344623566,
  -0.04364568740129471,
  -0.02344513311982155,
  0.013010554946959019,
  0.06681811064481735,
  -0.021194251254200935,
  0.073191635

##Now Lets import our vector store

In [ ]:
pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 7.1 MB/s eta 0:

In [ ]:
from langchain_chroma import Chroma
# Create a wrapper for the embedding model
class SentenceTransformerWrapper:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.embed_documents(texts)

    def embed_query(self, text):
        # Ensure text is passed as a list of strings
        return self.model.embed_documents([text])[0]
# Initialize HuggingFace model for embeddings with compatible dimension
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_wrapper = SentenceTransformerWrapper(embedding_model)

# Create the vector store with correct embeddings
db = Chroma.from_documents(data, embedding=embedding_wrapper)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now Search!

In [ ]:
 #Example query to test the vector store
query = "What is the company email of Sheryl Baxter?"
results = await db.asimilarity_search(query)
print("Query Results:", results)

Query Results: [Document(metadata={'row': 5, 'source': '/content/sample_data/sample.csv'}, page_content="title: Toyota Land\nprice: 71500000\nyear: 2022\nmileage: 7000\nfuel_type: Petrol\nhorsepower: 3500\ntransmission: Automatic\nlocation: Lahore\nregistration: Un-Registered\ncolor: Precious Wite Pearl\ndealer: PakWheels\nlisting_date: Oct 15, 2024\nfeatures: ABS,  AM/FM Radio,  Air Bags,  Air Conditioning,  Alloy Rims,  Cruise Control,  Immobilizer Key,  Keyless Entry,  Navigation System,  Power Locks,  Power Mirrors,  Power Steering,  Power Windows,  Sun Roof\nlink: https://www.pakwheels.com/used-cars/toyota-land-cruiser-2022-for-sale-in-lahore-9289663\nimages: ['670e55873c76c40f756a89c7']\npage: page 1\n_id: 670e55873c76c40f756a89c9\ndescription: Toyota Land, Petrol, Automatic, Lahore, Un-Registered, Precious Wite Pearl, ABS,  AM/FM Radio,  Air Bags,  Air Conditioning,  Alloy Rims,  Cruise Control,  Immobilizer Key,  Keyless Entry,  Navigation System,  Power Locks,  Power Mirrors, 

Checking ...

Checking the integrated model

In [ ]:
import getpass
import os
import asyncio
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage

# Set up the API key for ChatGroq
os.environ["GROQ_API_KEY"] = getpass.getpass()

# Initialize ChatGroq model
model = ChatGroq(model="llama3-8b-8192")

# Load your CSV file
file_path = r'/content/sample_data/sample.csv'
loader = CSVLoader(file_path=file_path)
data = loader.load()

# Initialize HuggingFace model for embeddings with compatible dimension
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a wrapper for the embedding model
class SentenceTransformerWrapper:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.embed_documents(texts)

    def embed_query(self, text):
         return self.model.embed_documents([text])[0]

embedding_wrapper = SentenceTransformerWrapper(embedding_model)

# Create the vector store with Sentence Transformers embeddings
db = Chroma.from_documents(data, embedding=embedding_wrapper)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a best salesman. Answer all user queries and try to catch the buyer. I will provide you the cars and you will tell the user about these details in an excited way"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Async function for node
async def call_model(state: MessagesState):
    chain = prompt | model
    response = await chain.ainvoke(state["messages"])
    return {"messages": response}

# Define the graph
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Define the async function to run the query and pass the data to the model
query = "Hi! My name is Sarab and i wanted to ask: Do you have an Audi A4?"
# Run similarity search
results = await db.asimilarity_search(query)

# Extract relevant information
context = " ".join(result.page_content for result in results)

# Create messages including the context
input_messages = [HumanMessage(query), HumanMessage(context)]

# Async invocation
config = {"configurable": {"thread_id": "abc123"}}
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()




··········


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


================================== Ai Message ==================================

WOOHOO! Welcome to the world of luxury, Sarab! I'm thrilled to introduce you to this stunning 2016 Audi A4, and I can assure you that this car will exceed your expectations in every way!

This sleek and powerful ride is a real head-turner, with its sleek Black exterior and premium interior. With a whopping 2000 horsepower under the hood, you'll feel like you're gliding on the roads of Lahore and Islamabad!

But it's not just about speed - this Audi A4 is also packed with incredible features that will make your daily commute a breeze. With ABS, Air Bags, Air Conditioning, and Power Steering, you'll be safe and comfortable no matter where you go. And with features like AM/FM Radio, CD Player, and Navigation System, you'll never be bored on the move!

But what truly sets this car apart is its low mileage of only 92,000 kilometers. That's a fraction of what most cars of this age would have, making it a rare f